In [9]:
user_mapping = {'bent': 0, 'kirsten': 1, 'ove': 2}
n_users = 3
n_recs = 3
recs = ['104', '102', '103', '101', '102', '104', '103', '101', '102']

cf_recs = {user_id: [] for user_id in user_mapping.keys()}
for i in range(n_users):
    for j in range(n_recs):
        rec = recs[i * n_recs + j]
        user_id = list(user_mapping.keys())[i]
        cf_recs[user_id].append(rec)
print(cf_recs)

{'bent': ['104', '102', '103'], 'kirsten': ['101', '102', '104'], 'ove': ['103', '101', '102']}


In [14]:
import json

# Data to be written
dictionary = {
	"name": "sathiyajith",
	"rollno": 56,
	"cgpa": 8.6,
	"phonenumber": "9976770500"
}

# Serializing json
json_object = json.dumps(dictionary, indent=4)

# Writing to sample.json
with open("sample.json", "w") as outfile:
	outfile.write(json_object)


In [30]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import utils
new_entry = {
    "user6": {"item8": 0.91}
}
utils.save_recommendations(new_entry, "test.json")


FileNotFoundError: [WinError 3] Den angivne sti blev ikke fundet: ''

In [ ]:
import pandas as pd

# importing training data
train_df = pd.read_parquet("../data/podcast_data_train.parquet")

# importing test data
test_df = pd.read_parquet("../data/podcast_data_test.parquet")

# grouping by user_id and counting the number of consumed items (not for completion_rate)
train_df_grouped = train_df.groupby("user_id")["prd_number"].count().reset_index()

# file path for the recommendations
file_path = "../results/recommendations.json"

# Open and load the JSON file
with open(file_path, "r") as file:
    data = json.load(file)

{'00018423377c14104870be7deda65daaa9dbc7b97f89a8d2f0702173d48e7739': {'11032421463',
  '11042408466',
  '11162220516',
  '11802401187'},
 '00051e4d15267dd420ebac00a1382db78d74e027c55d7f5db238822e29090e87': {'11032462201',
  '11032462202',
  '11802417401',
  '11802417402',
  '11802417403',
  '11802450463',
  '11802450464',
  '11802451186',
  '11802451187',
  '15452448045',
  '15452448048',
  '15452448049',
  '16122492639',
  '16122492640'},
 '000599ea041624d4e74e5ec2e285ae56abe4432eec717ba7d92dd8ebcb670579': {'11032419013',
  '11802450464',
  '11802451185',
  '11802451186',
  '11802451187',
  '11802451189',
  '11802451190',
  '11802451191',
  '11802451192',
  '11802451194',
  '11802451195',
  '11802451196',
  '11802451197'},
 '00060682bbc0cd8212733274e53e758949e624f477897016792e52114b647603': {'15452400031'},
 '00062a4d5b46ec9d8b39ef562b22374b74674e83eddce1f86717b16e8c258890': {'11032418467',
  '11802417401',
  '11802417403',
  '11802417405',
  '11802450462',
  '11802450484',
  '1180245

In [ ]:
# dictionary containing completion rates for each user
users = train_df["user_id"].unique()
completion_rate_dict = {user_id: {} for user_id in users}

for user_id in users:
    # filter the DataFrame for the current user
    user_df = train_df[train_df["user_id"] == user_id]
    # get the total number of items consumed by the user
    total_items = len(user_df)
    # get the number of unique items consumed by the user
    unique_items = len(user_df["prd_number"].unique())
    # calculate the completion rate
    completion_rate = unique_items / total_items if total_items > 0 else 0
    completion_rate_dict[user_id] = completion_rate

array(['00018423377c14104870be7deda65daaa9dbc7b97f89a8d2f0702173d48e7739',
       '00051e4d15267dd420ebac00a1382db78d74e027c55d7f5db238822e29090e87',
       '000599ea041624d4e74e5ec2e285ae56abe4432eec717ba7d92dd8ebcb670579',
       ...,
       'fffd0492116e42479d03ba2281dad9b9dcb4aaff8d852ddab1f5dd6be41fa606',
       'fffe6bcb1291f54947e534cc585ae11ba521c0cd07db9db327adf9e58c239696',
       'fffe6e1be686be369dfe0943282e377f7d1a51c0eae35a1f5033e325140c6f2d'],
      dtype=object)

In [ ]:
recommenders = ["cf_recommendations"]

for recommender in recommenders:
    # retrieving relevant recommendations
    recommendations = data["cf_recommendations"]

    # dictionaries to store evaluation metrics for each recommender
    recommender_dict = {}
    user_dict = {}

    # dictionary containing test items for each user
    test_items = {}
    for id, row in test_df.iterrows():
        if row["user_id"] not in test_items:
            test_items[row["user_id"]] = set()
        test_items[row["user_id"]].add(row["prd_number"])

    hit_dict = {user_id: 0 for user_id in recommendations.keys()}

    for user_id, rec_items in recommendations.items():
        rec_items = set(rec_items)
        true_items = test_items[user_id]
        correct_recs = rec_items.intersection(true_items)
        n_correct_recs = len(correct_recs)
        if n_correct_recs > 0:
            hit_dict[user_id] += 1
    
    # adding hit_dict to user_dict
    user_dict["hit_rate"] = hit_dict
    
    # calculating global hit rate
    hit_rate = sum(hit_dict.values()) / len(hit_dict)
    recommender_dict["hit_rate"] = hit_rate
    recommender_dict

    

In [48]:
recommender_dict

{'hit_rate': 0.0528708595903035}